In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import itertools
import mccd

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
def poly_pos(pos, max_degree, center_normalice=True,
             x_lims = None, y_lims = None):
    r"""Construct polynomial matrix.
    Return a matrix Pi containing polynomials of stars
    positions up to ``max_degree``.
    Defaulting to CFIS CCD limits.
    New method:
    The positions are scaled to the [-0.5, 0.5]x[-0.5, 0.5].
    Then the polynomials are constructed with the normalized positions.
    Old method:
    Positions are centred, the polynomials are constructed.
    Then the polynomials are normalized.
    """
    n_mono = (max_degree + 1) * (max_degree + 2) // 2
    Pi = np.zeros((n_mono, pos.shape[0]))
    _pos = np.copy(pos)
    if x_lims is None:
        x_min = np.min(_pos[:, 0])
        x_max = np.max(_pos[:, 0])
        x_lims = [x_min, x_max]
        # print('x_lims: ', x_lims)
    if y_lims is None:
        y_min = np.min(_pos[:, 1])
        y_max = np.max(_pos[:, 1])
        y_lims = [y_min, y_max]
        # print('y_lims: ', y_lims)
    if center_normalice:
        _pos[:, 0] = (_pos[:, 0] - x_lims[0])/(x_lims[1] - x_lims[0]) - 1/2
        _pos[:, 1] = (_pos[:, 1] - y_lims[0])/(y_lims[1] - y_lims[0]) - 1/2
    # print('min_x = ', np.min(_pos[:, 0]))     
    # print('max_x = ', np.max(_pos[:, 0]))
    # print('min_y = ', np.min(_pos[:, 1])) 
    # print('max_y = ', np.max(_pos[:, 1]))
    for d in range(max_degree + 1):
        row_idx = d * (d + 1) // 2
        for p in range(d + 1):
            Pi[row_idx + p, :] = _pos[:, 0] ** (d - p) * _pos[:, 1] ** p
    return Pi

In [3]:
def generate_params(poly=4, seed=None, ID=1):
    
    np.random.seed(seed=seed)
    params={}
    loc2glob = mccd.mccd_utils.Loc2Glob()
    
    max_x = loc2glob.x_npix*6 + loc2glob.x_gap*5
    min_x = loc2glob.x_npix*(-5) + loc2glob.x_gap*(-5)
    max_y = loc2glob.y_npix*2 + loc2glob.y_gap*1
    min_y = loc2glob.y_npix*(-2) + loc2glob.y_gap*(-2)
    
        
    coeff=[]
    pos1 = (max_x-min_x)*np.random.rand(100)+min_x
    pos2 = (max_y-min_y)*np.random.rand(100)+min_y
    pos = np.stack((pos1, pos2), axis=1)
    

    e1s = np.random.uniform(-0.12,0.12, (100, 1))
    e2s = np.random.uniform(-0.12,0.12, (100, 1))
    r2s = np.random.uniform(4,6, (100, 1))
        
            
    features = poly_pos(pos, poly, x_lims=[min_x, max_x], y_lims=[min_y, max_y])
    coeff_e1 = np.matmul(e1s.T,np.linalg.pinv(features))
    coeff_e2 = np.matmul(e2s.T,np.linalg.pinv(features))
    coeff_r2 = np.matmul(r2s.T,np.linalg.pinv(features))

    x = np.linspace(min_x, max_x, 100)
    y = np.linspace(min_y, max_y, 100)

    es = list(itertools.product(*[x, y]))
    x1 = np.array([a for a,b in es])
    y1 = np.array([b for a,b in es])
    pos = np.stack((x1, y1), axis=1)

    lim_e1 = np.random.uniform(0.05,0.075)
    lim_e2 = np.random.uniform(0.05,0.075)


    features = poly_pos(pos, poly, x_lims=[min_x, max_x], y_lims=[min_y, max_y])
    
    


    test_e1s = np.matmul(coeff_e1, features)
       
    params["max_e1"] = np.max(test_e1s)
    params["min_e1"] = np.min(test_e1s)
    params["coeffs_e1"] = coeff_e1
    params["lim_e1"] = lim_e1

    test_e1s = (test_e1s - np.min(test_e1s))/(np.max(test_e1s) - np.min(test_e1s))-1/2
    test_e1s = test_e1s*2*lim_e1
    
    
    

    test_e2s = np.matmul(coeff_e2, features)
    
    params["max_e2"] = np.max(test_e2s)
    params["min_e2"] = np.min(test_e2s)
    params["coeffs_e2"] = coeff_e2
    params["lim_e2"] = lim_e2

    test_e2s = (test_e2s - np.min(test_e2s))/(np.max(test_e2s) - np.min(test_e2s))-1/2
    test_e2s = test_e2s*2*lim_e2

    
    
    
    test_r2s = np.matmul(coeff_r2, features)
    
    params["max_r2"] = np.max(test_r2s)
    params["min_r2"] = np.min(test_r2s)
    params["coeffs_r2"] = coeff_r2

    test_r2s = (test_r2s - np.min(test_r2s))/((np.max(test_r2s) - np.min(test_r2s)))+4.5
    
    
    
    
    fig = plt.figure(num=0, figsize=(28,7))
    
    plt.subplot(131)
    
    plt.title("e1")
    plt.contourf(x, y, test_e1s[0].reshape(len(x),len(y)))
    plt.colorbar()
    
    plt.subplot(132)
    
    plt.title("e2")
    plt.contourf(x, y, test_e2s[0].reshape(len(x),len(y)))
    plt.colorbar()
    
    plt.subplot(133)
    
    plt.title("R2")
    plt.contourf(x, y, test_r2s[0].reshape(len(x),len(y)))
    plt.colorbar()
    
    plt.savefig('polynomials/coeffs_' + str(ID) + ".png")

    plt.clf()
    
    np.save('dict/params_' + str(ID) + '.npy', params)

In [4]:
for i in range (55):
    generate_params(poly=4, ID=i)

<Figure size 2016x504 with 0 Axes>

# Version to generate n samples of each parameter in separate dictionaries

In [145]:
def generate_e2(nb=10, poly=4, seed=1, unif=True):
    
    np.random.seed(seed=seed)
    coeffs={}
    loc2glob = mccd.mccd_utils.Loc2Glob()
    
    max_x = loc2glob.x_npix*6 + loc2glob.x_gap*5
    min_x = loc2glob.x_npix*(-5) + loc2glob.x_gap*(-5)
    max_y = loc2glob.y_npix*2 + loc2glob.y_gap*1
    min_y = loc2glob.y_npix*(-2) + loc2glob.y_gap*(-2)
    
    for i in range (nb):
        
        coeff=[]
        pos1 = (max_x-min_x)*np.random.rand(100)+min_x
        pos2 = (max_y-min_y)*np.random.rand(100)+min_y
        pos = np.stack((pos1, pos2), axis=1)
    
        if unif: 
            e1s = np.random.uniform(-0.12,0.12, (100, 1))
        else: 
                e1s = np.random.normal(0,0.055, (100, 1))
            
        features = poly_pos(pos, poly, x_lims=[min_x, max_x], y_lims=[min_y, max_y])
        coeff = np.matmul(e1s.T,np.linalg.pinv(features))
    
        x = np.linspace(min_x, max_x, 100)
        y = np.linspace(min_y, max_y, 100)

        es = list(itertools.product(*[x, y]))
        x1 = np.array([a for a,b in es])
        y1 = np.array([b for a,b in es])
        lim_e1 = np.random.uniform(0.05,0.075)

        pos = np.stack((x1, y1), axis=1)
        features = poly_pos(pos, poly, x_lims=[min_x, max_x], y_lims=[min_y, max_y])
        test_e1s = np.matmul(coeff, features)
        coeffs["max_e2_" + str(i)] = np.max(test_e1s)
        coeffs["min_e2_" + str(i)] = np.min(test_e1s)
        #coeff = coeff*(0.06/np.max(test_e1s))
        test_e1s = (test_e1s - np.min(test_e1s))/(np.max(test_e1s) - np.min(test_e1s))-1/2
        test_e1s = test_e1s*2*lim_e1
    
        coeffs["coeffs_e2_" + str(i)] = coeff
        

        plt.contourf(x, y, test_e1s[0].reshape(len(x),len(y)))
        plt.colorbar()
        
        plt.savefig('e2/coeffs_' + str(i) + ".png")
        plt.clf()
    
    return coeffs

def generate_r2(nb=10, poly=4, seed=1, unif=True):
    
    np.random.seed(seed=seed)
    coeffs={}
    loc2glob = mccd.mccd_utils.Loc2Glob()
    
    max_x = loc2glob.x_npix*6 + loc2glob.x_gap*5
    min_x = loc2glob.x_npix*(-5) + loc2glob.x_gap*(-5)
    max_y = loc2glob.y_npix*2 + loc2glob.y_gap*1
    min_y = loc2glob.y_npix*(-2) + loc2glob.y_gap*(-2)
    
    for i in range (nb):
        
        coeff=[]
        pos1 = (max_x-min_x)*np.random.rand(100)+min_x
        pos2 = (max_y-min_y)*np.random.rand(100)+min_y
        pos = np.stack((pos1, pos2), axis=1)
    
        if unif: 
            e1s = np.random.uniform(4,6, (100, 1))
        else: 
                e1s = np.random.normal(0,0.055, (100, 1))
            
        features = poly_pos(pos, poly, x_lims=[min_x, max_x], y_lims=[min_y, max_y])
        coeff = np.matmul(e1s.T,np.linalg.pinv(features))
    
        x = np.linspace(min_x, max_x, 100)
        y = np.linspace(min_y, max_y, 100)

        es = list(itertools.product(*[x, y]))
        x1 = np.array([a for a,b in es])
        y1 = np.array([b for a,b in es])
        lim_e1 = np.random.uniform(0.05,0.075)

        pos = np.stack((x1, y1), axis=1)
        features = poly_pos(pos, poly, x_lims=[min_x, max_x], y_lims=[min_y, max_y])
        test_e1s = np.matmul(coeff, features)
        coeffs["max_r2_" + str(i)] = np.max(test_e1s)
        coeffs["min_r2_" + str(i)] = np.min(test_e1s)
        #coeff = coeff*(0.06/np.max(test_e1s))
        test_e1s = (test_e1s - np.min(test_e1s))/((1/2)*(np.max(test_e1s) - np.min(test_e1s)))+4
    
        coeffs["coeffs_r2_" + str(i)] = coeff
        

        plt.contourf(x, y, test_e1s[0].reshape(len(x),len(y)))
        plt.colorbar()
        
        plt.savefig('R2/coeffs_'+ str(i) + ".png")
        plt.clf()
    
    return coeffs



